In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# matplotlib.use("Agg")

from ase.io import read
from ase.build import bulk
from raffle.generator import raffle_generator
from mace.calculators import mace_mp
# from agox.utils.replica_exchange.priors import get_prior

import numpy as np

In [ ]:
## Set up the plotting environment
# matplotlib.rcParams.update(matplotlib.rcParamsDefault)
plt.rc('text', usetex=True)
plt.rc('font', family='cmr10', size=12)
plt.rcParams["axes.formatter.use_mathtext"] = True

In [ ]:
calc_params = { 'model':  '../mace-mpa-0-medium.model' }
calc = mace_mp(**calc_params)

In [ ]:
Si_bulk = bulk("Si", crystalstructure="diamond", a=5.43)
Si_bulk.calc = calc
Si_reference_energy = Si_bulk.get_potential_energy() / len(Si_bulk)
Ge_bulk = bulk("Ge", crystalstructure="diamond", a=5.65)
Ge_bulk.calc = calc
Ge_reference_energy = Ge_bulk.get_potential_energy() / len(Ge_bulk)

element_energies = {'Si': Si_reference_energy, 'Ge': Ge_reference_energy}

In [ ]:
host = read("host.traj")
host.calc = calc
structures = read("converted_db.traj", index=":")

In [ ]:
generator = raffle_generator(
    history_len = 10,
    host = host,
    element_energies = element_energies,
    kBT = 0.2
)

In [ ]:
generator.distributions.create([Si_bulk, Ge_bulk])

In [ ]:
for i, structure in enumerate(structures):
    print(structures[:i+1])

In [ ]:
convergence = []
for i, structure in enumerate(structures):
    generator.distributions.update(structure)
    convergence.append( generator.distributions.history_deltas[0])


In [ ]:
from matplotlib.scale import ScaleBase
from matplotlib.transforms import Transform
import numpy as np

class CubeRootScale(ScaleBase):
    name = 'cuberoot'

    def get_transform(self):
        return self.CubeRootTransform()

    def set_default_locators_and_formatters(self, axis):
        axis.set_major_locator(plt.MaxNLocator(integer=True))
        axis.set_major_formatter(plt.FuncFormatter(lambda y, _: f"{y**3:.2f}"))

    class CubeRootTransform(Transform):
        input_dims = output_dims = 1

        def transform_non_affine(self, a):
            a = np.asarray(a)
            return np.cbrt(a)

        def inverted(self):
            return CubeRootScale.InvertedCubeRootTransform()

    class InvertedCubeRootTransform(Transform):
        input_dims = output_dims = 1

        def transform_non_affine(self, a):
            a = np.asarray(a)
            return a #**3

        def inverted(self):
            return CubeRootScale.CubeRootTransform()

# Register the custom scale
import matplotlib.scale as mscale
mscale.register_scale(CubeRootScale)


In [ ]:
# plot convergence log plot as a function of the number of structures
# ax = plt.figure(figsize=(8, 6))
%matplotlib widget
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)
power = 1.0 #/ 3.0
plt.plot(np.arange(len(convergence)), [ x**power for x in convergence], color="black", linestyle="-", marker='o', markersize=3, linewidth=0.5, label='Convergence')
# change colour of line to black

# add a window averaged line
window_size = 10
averaged = np.convolve([ x**power for x in convergence], np.ones(window_size)/window_size, mode='valid')
plt.plot(np.arange(window_size-1, len(convergence)), averaged, color='red', label='Window Averaged')
plt.legend()

# add a cumulative average line
cumulative_average = np.cumsum([ x**power for x in convergence]) / np.arange(1, len(convergence)+1)
plt.plot(np.arange(len(convergence)), cumulative_average, color='blue', label='Cumulative Average')
plt.legend()

ax.set_xlabel('Number of Structures', fontsize=20)
ax.set_ylabel('Convergence Metric', fontsize=20)
# plt.title('Convergence of the Raffle Generator')
plt.show()

# set x and y limits
ax.set_xlim(0, len(convergence)+1)
ax.set_ylim(0, 1)

# add y=0 line
ax.axhline(0, color='black', linestyle='--', linewidth=1.0)



ax.set_yscale('cuberoot')
# # Desired original y-values (before cube root)
# --- Custom ticks in original (linear) space ---
major_linear = [0.0, 0.01, 0.1, 1.0]

minor_linear = []
for major in major_linear:
    minor_linear.extend([ major * 0.1 * i  for i in range(1, 10)])

# Convert to cube root space
major_cuberoot = [x**power for x in major_linear]
minor_cuberoot = [x**power for x in minor_linear]

# Set major and minor ticks
ax.set_yticks(major_cuberoot)
ax.set_yticklabels([f"{x:.2f}" for x in major_linear])
ax.set_yticks(minor_cuberoot, minor=True)


# change x ticks
ax.xaxis.set_major_locator(plt.MultipleLocator(50))
ax.xaxis.set_minor_locator(plt.MultipleLocator(25))

# have the ticks point intwards and on both sides
ax.tick_params(axis='both', which='major', direction='in', length=10, width=1)
ax.tick_params(axis='both', which='minor', direction='in', length=5, width=1)
ax.tick_params(axis='x', which='both', bottom=True, top=True)
ax.tick_params(axis='y', which='both', left=True, right=True)

# set the legend font size
for label in (ax.get_xticklabels() + ax.get_yticklabels()):
    label.set_fontsize(16)
ax.legend(fontsize=16)

plt.savefig('convergence.pdf', bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor(), edgecolor='none')